In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Load and preprocess data
df = pd.read_csv('hottest_50_1960_2022_encoding.csv')
df = df.dropna(subset=['danceability', 'energy', 'loudness', 'speechiness', 'valence', 'tempo'])  # Exclude missing rows
df['year'] = df['date'].str[:4].astype(int)
df['decade'] = (df['year'] // 10) * 10  # Group into decades

# Features for clustering
features = ['danceability', 'energy', 'loudness', 'speechiness', 'valence', 'tempo']
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[features])

# Determine optimal clusters (Elbow Method)
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(df_scaled)
    wcss.append(kmeans.inertia_)
optimal_clusters = 4  # Adjust based on elbow curve

In [ ]:
# Initialize Dash app
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Dynamic Genre Evolution (1960-2020)", style={'textAlign': 'center'}),

    html.Div([
        dcc.Dropdown(
            id='decade-selector',
            options=[{'label': f'{decade}s', 'value': decade} for decade in sorted(df['decade'].unique())],
            value=df['decade'].min(),
            placeholder="Select Decade",
            style={'width': '100%'}
        )
    ], style={'width': '50%', 'margin': '20px auto'}),

    dcc.Graph(id='cluster-plot'),
    dcc.Graph(id='heatmap'),

    html.H3("Select Feature for Evolution Comparison:", style={'textAlign': 'center'}),

    html.Div([
        dcc.Dropdown(
            id='feature-selector',
            options=[{'label': feat.title(), 'value': feat} for feat in features],
            value='danceability',
            placeholder="Select Feature",
            style={'width': '100%'}
        )
    ], style={'width': '50%', 'margin': '20px auto'}),

    dcc.Graph(id='feature-evolution')
])


In [ ]:
# Callbacks for interactivity
@app.callback(
    [Output('cluster-plot', 'figure'),
     Output('heatmap', 'figure'),
     Output('feature-evolution', 'figure')],
    [Input('decade-selector', 'value'),
     Input('feature-selector', 'value')]
)
def update_plots(selected_decade, selected_feature):
    # Filter data by decade
    df_decade = df[df['decade'] == selected_decade].copy()
    scaled_data = scaler.transform(df_decade[features])

    # K-Means Clustering
    kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
    df_decade['cluster'] = kmeans.fit_predict(scaled_data)

    # t-SNE Projection
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=500)
    tsne_results = tsne.fit_transform(scaled_data)
    df_decade['tsne_x'] = tsne_results[:, 0]
    df_decade['tsne_y'] = tsne_results[:, 1]

    # Cluster vs Genre Heatmap
    heatmap_data = pd.crosstab(df_decade['cluster'], df_decade['genre_encoding'])
    heatmap_fig = px.imshow(heatmap_data, labels=dict(x="Genre", y="Cluster", color="Count"),
                            title=f"Cluster vs Genre (Decade: {selected_decade}s)")

    # Feature Evolution (2000s vs 2020s)
    df_2000s = df[df['decade'] == 2000]
    df_2020s = df[df['decade'] == 2020]
    feature_evolution_fig = go.Figure()

    feature_evolution_fig.add_trace(go.Box(
        x=df_2000s['genre_encoding'],
        y=df_2000s[selected_feature],
        name=f'2000s {selected_feature.title()}'
    ))

    feature_evolution_fig.add_trace(go.Box(
        x=df_2020s['genre_encoding'],
        y=df_2020s[selected_feature],
        name=f'2020s {selected_feature.title()}'
    ))

    feature_evolution_fig.update_layout(
        title=f"Feature Evolution: {selected_feature.title()} (2000s vs 2020s)",
        xaxis_title="Genre Encoding",
        yaxis_title=selected_feature.title()
    )

    # Cluster Plot
    cluster_fig = px.scatter(
        df_decade, x='tsne_x', y='tsne_y', color='cluster',
        hover_data=['title', 'artist', 'genre_encoding'],
        title=f"Clusters in the {selected_decade}s (t-SNE Projection)"
    )

    return cluster_fig, heatmap_fig, feature_evolution_fig


In [ ]:
# Run the app in Colab
app.run(mode='inline', port=8050)

<IPython.core.display.Javascript object>